In [42]:
# This notebook will be to create a stock screener

from config.secrets import MYSQL_HOST, MYSQL_PORT, MYSQL_PASSWORD, MYSQL_USER, MYSQL_MARKET_DB, MYSQL_PRICEHISTORY_DB
import mysql.connector as mysql
from mysql.connector import Error
import requests, os
import pandas as pd
import numpy as np
from loguru import logger
import yfinance as yf

In [43]:
# =============================================================================== #
# CREATE THE LOGGER FOR THIS SCRIPT:
# =============================================================================== #
log_path = str(os.path.pardir) + '/logs/'
base_fmt = "[{time:YYYY-MM-DD at HH:mm:ss}]|[{name}-<lvl>{message}</lvl>]"
logger.add(log_path+"main.log", rotation="2 MB", colorize=True, enqueue=True, catch=True)

4

In [48]:
# Just incase I decide to use the RapidAPI yahoo api to get the balance sheet data 
# I kept the information in here

rapidAPI_yahoo_headers = {
		"content-type": "application/x-www-form-urlencoded",
		"X-RapidAPI-Key": "292e5f6756msh0abb3103cb293b6p189764jsn96d2fc366bde",
		"X-RapidAPI-Host": "yahoo-finance97.p.rapidapi.com"
	}

url = "https://yahoo-finance97.p.rapidapi.com/financials"

config = {
	'host': MYSQL_HOST,
	'port': MYSQL_PORT,
	'database': MYSQL_MARKET_DB,
	'user': 'root',
	'password': MYSQL_PASSWORD,
	'use_unicode': True,
	'get_warnings': True,
}

In [52]:
MYSQL_PASSWORD

'MZMfib112358!'

In [50]:
def _select_symbols(config:dict):
    """Selects rows from table. Give DB name, table name, and
    num of rows to select and display"""
    table = 'companies'
    query_symbols = f"SELECT symbol FROM {table}"
    try:
        symbols = []
        with mysql.connect(**config) as conn:
            with conn.cursor() as cursor:
                cursor.execute(query_symbols)
                result = cursor.fetchall()
                for row in result:
                    symbols.append(row[0])
        return symbols
    except Error as e:
        logger.error("[-] Error {} occured", e)


In [51]:
symbols = _select_symbols(config)

2022-08-13 12:22:24.581 | ERROR    | __main__:_select_symbols:16 - [-] Error 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES) occured
